# Data Loading

In [1]:
spark.conf.set("spark.sql.shuffle.partitions", 1000)
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1)

In [2]:
from pyspark.sql.types import DateType, IntegerType
from pyspark.sql.functions import from_unixtime, unix_timestamp,to_date, max, when
from pyspark.sql.functions import col, sum, greatest, expr, count, countDistinct, concat,substring, lit, length, lpad
from pyspark.sql.types import *
from pyspark.sql.functions import broadcast
#import datetime

In [3]:
#FUNCIONES PARA EXTRAER NULOS POR COLUMNA
def count_null(col_name):
  return sum(col(col_name).isNull().cast('integer')).alias(col_name)

def mostrarNulos(dataframe):
    # Run the aggregation. The *exprs converts the list of expressions into
    # variable function arguments.
    exprs = []
    for col_name in dataframe.columns:
      exprs.append(count_null(col_name))
    return dataframe.agg(*exprs)

In [4]:
#PARA LOCAL DEL TRABAJO
#spark = SparkSession.builder.getOrCreate()

## Buffer Ticket

#### PRODUTOFAM

In [4]:
#LECTURA DE TABLA PARQUET
dfProdutofam=spark.read.parquet("/user/sparta-server/trusted/bufferticket/PRODUTOFAM")
dfProdutofam.persist()

DataFrame[PRF_CD_SUBFAMILIA: bigint, PRF_DS_SUBFAMILIA: string, PRF_CD_FAMILIA: bigint, PRF_DT_INCLUSAO: timestamp, PRF_DT_ALTERACAO: timestamp, TM_TRUSTED: timestamp, DT_TRUSTED: int]

In [5]:
dfProdutofam.printSchema()

root
 |-- PRF_CD_SUBFAMILIA: long (nullable = true)
 |-- PRF_DS_SUBFAMILIA: string (nullable = true)
 |-- PRF_CD_FAMILIA: long (nullable = true)
 |-- PRF_DT_INCLUSAO: timestamp (nullable = true)
 |-- PRF_DT_ALTERACAO: timestamp (nullable = true)
 |-- TM_TRUSTED: timestamp (nullable = true)
 |-- DT_TRUSTED: integer (nullable = true)



In [ ]:
dfProdutofam.describe().show()

In [14]:
#DETECTAR DUPLICADOS
dfProdutofam.count()

3076

In [7]:
dfProdutofam.select("PRF_CD_SUBFAMILIA").distinct().count()

3076

In [13]:
dfProdutofam=dfProdutofam.dropDuplicates(["PRF_CD_SUBFAMILIA"])

In [ ]:
#CONVERSION A FECHA, PARA CUANDO SEA NECESARIO

#dfProdutofam.select("PRF_DT_INCLUSAO").cast(DateType)
#dfProdutofam.withColumn("record_date",dfProdutofam['PRF_DT_INCLUSAO'].cast(DateType())).show()
#dfProdutofam.withColumn("parsed", to_date(to_timestamp("PRF_DT_INCLUSAO", "dd/MM/yy"))).show()
dfProdutofam.withColumn("parsed", to_date(from_unixtime(unix_timestamp("PRF_DT_INCLUSAO", "dd/MM/yy")))).show()

#ESCRITURA A PARQUET

#df.write.parquet("testparquet")
#df2= spark.read.parquet("testparquet")

In [ ]:
mostrarNulos(dfProdutofam).show()

#### PRODUTOFAMDEP

In [15]:
#LECTURA DE TABLA PARQUET
dfProdutofamdep=spark.read.parquet("/user/sparta-server/trusted/bufferticket/PRODUTOFAMDEP")
dfProdutofamdep.persist()

DataFrame[PRF_CD_DEPARTAMENTO: int, PRF_DS_DEPARTAMENTO: string, PRF_DT_INCLUSAO: timestamp, PRF_DT_ALTERACAO: timestamp, TM_TRUSTED: timestamp, DT_TRUSTED: int]

In [16]:
dfProdutofamdep.printSchema()

root
 |-- PRF_CD_DEPARTAMENTO: integer (nullable = true)
 |-- PRF_DS_DEPARTAMENTO: string (nullable = true)
 |-- PRF_DT_INCLUSAO: timestamp (nullable = true)
 |-- PRF_DT_ALTERACAO: timestamp (nullable = true)
 |-- TM_TRUSTED: timestamp (nullable = true)
 |-- DT_TRUSTED: integer (nullable = true)



In [20]:
#DETECTAR DUPLICADOS
dfProdutofamdep.count()

9

In [18]:
dfProdutofamdep.select("PRF_CD_DEPARTAMENTO").distinct().count()

9

In [19]:
dfProdutofamdep=dfProdutofamdep.dropDuplicates(["PRF_CD_DEPARTAMENTO"])

In [ ]:
dfProdutofamdep.describe().show()

In [ ]:
mostrarNulos(dfProdutofamdep).show()

#### PRODUTOFAMFAM

In [8]:
#LECTURA DE TABLA PARQUET
dfProdutofamfam=spark.read.parquet("/user/sparta-server/trusted/bufferticket/PRODUTOFAMFAM")
dfProdutofamfam.persist()

DataFrame[PRF_CD_FAMILIA: bigint, PRF_DS_FAMILIA: string, PRF_CD_GRUPO: bigint, PRF_DT_INCLUSAO: timestamp, PRF_DT_ALTERACAO: timestamp, TM_TRUSTED: timestamp, DT_TRUSTED: int]

In [ ]:
dfProdutofamfam.printSchema()

In [ ]:
#DETECTAR DUPLICADOS
dfProdutofamfam.count()==dfProdutofamfam.distinct().count()

In [ ]:
dfProdutofamfam.describe().show()

In [12]:
dfProdutofamfam.count()

3079

In [10]:
dfProdutofamfam.select("PRF_CD_FAMILIA").distinct().count()

3079

In [11]:
dfProdutofamfam=dfProdutofamfam.dropDuplicates(["PRF_CD_FAMILIA"])

In [ ]:
dfProdutofamfam.groupBy("PRF_CD_FAMILIA").count().where(col('count')>1).show(50)

In [ ]:
mostrarNulos(dfProdutofamfam).show()

#### PRODUTOFAMGRP

In [21]:
#LECTURA DE TABLA PARQUET
dfProdutofamgrp=spark.read.parquet("/user/sparta-server/trusted/bufferticket/PRODUTOFAMGRP")
dfProdutofamgrp.persist()

DataFrame[PRF_CD_GRUPO: bigint, PRF_DS_GRUPO: string, PRF_CD_SETOR: int, PRF_DT_INCLUSAO: timestamp, PRF_DT_ALTERACAO: timestamp, TM_TRUSTED: timestamp, DT_TRUSTED: int]

In [22]:
dfProdutofamgrp.printSchema()

root
 |-- PRF_CD_GRUPO: long (nullable = true)
 |-- PRF_DS_GRUPO: string (nullable = true)
 |-- PRF_CD_SETOR: integer (nullable = true)
 |-- PRF_DT_INCLUSAO: timestamp (nullable = true)
 |-- PRF_DT_ALTERACAO: timestamp (nullable = true)
 |-- TM_TRUSTED: timestamp (nullable = true)
 |-- DT_TRUSTED: integer (nullable = true)



In [ ]:
#DETECTAR DUPLICADOS
dfProdutofamgrp.count()==dfProdutofamgrp.distinct().count()

In [ ]:
dfProdutofamgrp.count()

In [23]:
dfProdutofamgrp.select("PRF_CD_GRUPO").distinct().count()

335

In [25]:
dfProdutofamgrp.count()

335

In [24]:
dfProdutofamgrp=dfProdutofamgrp.dropDuplicates(["PRF_CD_GRUPO"])

In [ ]:
groupList=[i.PRF_CD_GRUPO for i in dfProdutofamgrp.select('PRF_CD_GRUPO').distinct().orderBy('PRF_CD_GRUPO').collect()]

In [ ]:
len(groupList)

In [ ]:
dfProdutofamgrp.describe().show()

In [ ]:
mostrarNulos(dfProdutofamgrp).show()

#### PRODUTOFAMSET

In [26]:
#LECTURA DE TABLA PARQUET
dfProdutofamset=spark.read.parquet("/user/sparta-server/trusted/bufferticket/PRODUTOFAMSET")
dfProdutofamset.persist()

DataFrame[PRF_CD_SETOR: int, PRF_DS_SETOR: string, PRF_CD_DEPARTAMENTO: int, PRF_DT_INCLUSAO: timestamp, PRF_DT_ALTERACAO: timestamp, TM_TRUSTED: timestamp, DT_TRUSTED: int]

In [ ]:
dfProdutofamset.printSchema()

In [ ]:
#DETECTAR DUPLICADOS
dfProdutofamset.count()==dfProdutofamset.distinct().count()

In [30]:
dfProdutofamset.count()

40

In [28]:
dfProdutofamset.select("PRF_CD_SETOR").distinct().count()

40

In [29]:
dfProdutofamset=dfProdutofamset.dropDuplicates(["PRF_CD_SETOR"])

In [ ]:
dfProdutofamset.describe().show()

In [ ]:
mostrarNulos(dfProdutofamset).show()

#### ITEM_VAREJO

In [31]:
#LECTURA DE TABLA PARQUET
dfItemvarejo=spark.read.parquet("/user/sparta-server/trusted/bufferticket/ITEM_VAREJO")
dfItemvarejo.persist()

DataFrame[IVA_CD_ITEM: bigint, PRF_CD_SUBFAMILIA: bigint, FOR_CD_FORNECEDOR: bigint, IVA_DS_ITEM: string, IVA_DT_INSERT: timestamp, IVA_DT_UPDATE: timestamp, TM_TRUSTED: timestamp, DT_TRUSTED: int]

In [42]:
dfItemvarejo.printSchema()

root
 |-- IVA_CD_ITEM: long (nullable = true)
 |-- PRF_CD_SUBFAMILIA: long (nullable = true)
 |-- FOR_CD_FORNECEDOR: long (nullable = true)
 |-- IVA_DS_ITEM: string (nullable = true)
 |-- IVA_DT_INSERT: timestamp (nullable = true)
 |-- IVA_DT_UPDATE: timestamp (nullable = true)
 |-- TM_TRUSTED: timestamp (nullable = true)
 |-- DT_TRUSTED: integer (nullable = true)



In [ ]:
#DETECTAR DUPLICADOS
dfItemvarejo.count()==dfItemvarejo.distinct().count()

In [ ]:
dfItemvarejo=dfItemvarejo.dropDuplicates()

In [35]:
dfItemvarejo.count()

456658

In [33]:
dfItemvarejo.select("IVA_CD_ITEM").distinct().count()

456658

In [34]:
dfItemvarejo=dfItemvarejo.dropDuplicates(["IVA_CD_ITEM"])

In [ ]:
dfItemvarejo.describe().show()

In [ ]:
mostrarNulos(dfItemvarejo).show()

#### ITEM_CUPOM

In [13]:
#LECTURA DE TABLA PARQUET
dfItemcupom=spark.read.parquet("/user/sparta-server/trusted/bufferticket/ITEM_CUPOM")
# dfItemcupom=spark.read.parquet("/bkp/ITEM_CUPOM")

In [14]:
dfItemcupom.printSchema()

root
 |-- CUP_ID_CUPOM: long (nullable = true)
 |-- IVA_CD_ITEM: long (nullable = true)
 |-- CUP_NU_CUPOM: integer (nullable = true)
 |-- CUP_DT_CUPOM: timestamp (nullable = true)
 |-- ICU_QT_ITECOMPRADOS: double (nullable = true)
 |-- ICU_VL_ITENS: double (nullable = true)
 |-- ICU_FG_PROMOCAO: string (nullable = true)
 |-- ICU_FG_ITEPESADO: string (nullable = true)
 |-- ICU_DT_INSERT: timestamp (nullable = true)
 |-- ICU_DT_UPDATE: timestamp (nullable = true)
 |-- TM_TRUSTED: timestamp (nullable = true)
 |-- DT_TRUSTED: integer (nullable = true)



In [46]:
dfItemcupom.count()
#3171783077
#3176595289

3176595289

In [ ]:
dfItemcupom.describe().show()

In [ ]:
mostrarNulos(dfItemcupom).show()

#### CUPOM

In [15]:
#LECTURA DE TABLA PARQUET
dfCupom=spark.read.parquet("/user/sparta-server/trusted/bufferticket/CUPOM")

In [16]:
dfCupom.printSchema()

root
 |-- CUP_ID_CUPOM: long (nullable = true)
 |-- TRA_CD_TRANSACAO: long (nullable = true)
 |-- CUP_NU_CUPOM: integer (nullable = true)
 |-- CUP_CD_ESTABELECIMENTO: long (nullable = true)
 |-- CUP_CD_PDV: string (nullable = true)
 |-- CUP_DT_CUPOM: timestamp (nullable = true)
 |-- CUP_HR_CUPOM: integer (nullable = true)
 |-- CUP_VL_CUPOM: double (nullable = true)
 |-- CUP_DT_INSERT: timestamp (nullable = true)
 |-- CUP_DT_UPDATE: timestamp (nullable = true)
 |-- PES_NU_CPFCNPJ: long (nullable = true)
 |-- SITE_COD: integer (nullable = true)
 |-- FISCAL_COUPON_COD: integer (nullable = true)
 |-- TRSACT_COD: integer (nullable = true)
 |-- SALE_TERMNL_COD: integer (nullable = true)
 |-- CUP_VL_TROCO: double (nullable = true)
 |-- TM_TRUSTED: timestamp (nullable = true)
 |-- DT_TRUSTED: integer (nullable = true)



In [17]:
dfCupom.count()

519187849

In [49]:
dfCupom.select('CUP_ID_CUPOM').distinct().count()

514428341

In [ ]:
dfCupom.describe().show()

In [ ]:
mostrarNulos(dfCupom).toPandas()

In [ ]:
#dfCupom.select("PES_NU_CPFCNPJ").show(10)

#### TRANSACAO

In [18]:
#LECTURA DE TABLA PARQUET
dfTransacao=spark.read.parquet("/user/sparta-server/trusted/bufferticket/TRANSACAO")

In [19]:
dfTransacao.printSchema()

root
 |-- TRA_CD_TRANSACAO: long (nullable = true)
 |-- TTR_CD_TIPTRANSACAO: integer (nullable = true)
 |-- CTA_CD_CONTA: long (nullable = true)
 |-- MTR_CD_MEITRANSACAO: string (nullable = true)
 |-- EST_CD_ESTABELECIMENTO: long (nullable = true)
 |-- PLA_CD_PLANO: long (nullable = true)
 |-- CUP_NU_CUPOM: long (nullable = true)
 |-- PRD_CD_PROORIGINAL: string (nullable = true)
 |-- TRA_CD_NATTRANSACAO: string (nullable = true)
 |-- TRA_DT_TRANSACAO: timestamp (nullable = true)
 |-- TRA_HR_TRANSACAO: string (nullable = true)
 |-- TRA_DT_EFETRANSACAO: timestamp (nullable = true)
 |-- TRA_VL_TRANSACAO: double (nullable = true)
 |-- TRA_VL_COMISSAO: double (nullable = true)
 |-- TRA_VL_LIQTRANSACAO: double (nullable = true)
 |-- TRA_VL_PARC_1: double (nullable = true)
 |-- TRA_VL_PARC_DEMAIS: double (nullable = true)
 |-- TRA_TP_MOETRANSACAO: string (nullable = true)
 |-- TRA_QT_PARCELAS: integer (nullable = true)
 |-- TRA_NU_AUTORIZACAO: long (nullable = true)
 |-- TRA_NU_NSU: string (n

In [51]:
dfTransacao.count()

470287150

In [52]:
dfTransacao.select("TRA_CD_TRANSACAO").distinct().count()

470287141

In [ ]:
dfTransacao.describe().show()

In [ ]:
mostrarNulos(dfTransacao).show()

## TSYS

####  CodeProducts

In [20]:
#ARCHIVO CREADO A MANO CON LOS CÓDIGOS
dfCodeproducts = spark.read \
    .option("header", "true").option("inferSchema","true").option("delimiter", ",").csv("/analytics/bc1_clustering/inputs/Codeproducts2.csv")
    #.option("header", "false").option("inferSchema","true").option("delimiter", "|").csv("file:///C:\\Users\Fabian de Plasencia\Documents\Business Case 1\sample_BC_TSYS_CACCOUNTS.txt")
    #.option("header", "true").option("inferSchema","true").option("delimiter", "|").csv("file:///C:\Users\Fabiande\Documents\Business Case 1\BC_PRODUTOFAMDEP.txt")

In [21]:
dfCodeproducts.printSchema()

root
 |-- SERNO: integer (nullable = true)
 |-- CD_GL: string (nullable = true)
 |-- DESCRICAO: string (nullable = true)
 |-- TIPO_TRNS: string (nullable = true)
 |-- TIPO_TRNS_N2: string (nullable = true)
 |-- FG_TRNS: string (nullable = true)
 |-- FG_ONOFF: string (nullable = true)
 |-- CICLO: string (nullable = true)
 |-- CORTE: string (nullable = true)
 |-- JUROS: string (nullable = true)
 |-- MEIOPAGAM: string (nullable = true)
 |-- FG_ONOFFCASE: string (nullable = true)
 |-- MC: string (nullable = true)
 |-- MC2: string (nullable = true)
 |-- VISA: string (nullable = true)
 |-- VISA2: string (nullable = true)
 |-- CHIP: string (nullable = true)
 |-- CRIADA_EM: string (nullable = true)
 |-- ATUALIZADA_EM: string (nullable = true)
 |-- ORIGEM: string (nullable = true)
 |-- LOCALCOMPRA: string (nullable = true)
 |-- PRODUTO: string (nullable = true)
 |-- TIPO_TRANSACAO: string (nullable = true)



In [22]:
dfCodeproducts.select('PRODUTO').distinct().show(300, truncate=False)

+------------------------------+
|PRODUTO                       |
+------------------------------+
|Saque_Parcelado               |
|Emprestimo_Pessoal            |
|Parcelamento_Fatura_Automatico|
|Overlimit                     |
|Off_Us                        |
|Pagamento_de_Contas           |
|Saque_a_Vista                 |
|Parcelamento_Fatura_Facil     |
+------------------------------+



In [ ]:
mostrarNulos(dfCodeproducts).show()

#### ACCOUNTS

In [34]:
#LECTURA DE TABLA PARQUET ANALYTICS
# dfAccounts=spark.read.parquet("/analytics/bc1_clustering/tables/dfAccounts")
dfAccounts=spark.read.parquet("/analytics/bc1_clustering/tables/dfAccountsRefdate")
dfAccounts=dfAccounts.withColumnRenamed("SERNO","ACCOUNTSERNO").withColumnRenamed("PEOPLESERNO","PEOPLESERNOFK")

In [ ]:
#LECTURA DE TABLA PARQUET TRUSTED
dfAccounts=spark.read.parquet("/user/sparta-server/trusted/tsys/CACCOUNTS")

In [ ]:
dfAccounts.explain()

In [ ]:
dfAccounts.printSchema()

In [ ]:
# dfAccountsMax=dfAccounts.groupBy('SERNO').agg(max('REFDATE').alias('FECHAMAX'))
#QUITANDO PERIODO DE VALIDACION
dfAccountsMax=dfAccounts.where(to_date(col('REFDATE'))<'2017-11-01') \
    .groupBy('SERNO').agg(max('REFDATE').alias('FECHAMAX'))

In [ ]:
dfAccounts2=dfAccounts.join(dfAccountsMax,(dfAccountsMax.SERNO==dfAccounts.SERNO) & (dfAccountsMax.FECHAMAX==dfAccounts.REFDATE),'leftsemi')

# dfAccounts2=dfAccounts.join(dfAccountsMax,(dfAccountsMax.SERNO==dfAccounts.SERNO) & (dfAccountsMax.FECHAMAX==dfAccounts.REFDATE)) \
#     .drop(dfAccountsMax.SERNO).drop(dfAccountsMax.FECHAMAX)


dfAccounts2.printSchema()

In [ ]:
dfAccounts=dfAccounts2.withColumnRenamed("SERNO","ACCOUNTSERNO").withColumnRenamed("PEOPLESERNO","PEOPLESERNOFK")

In [ ]:
dfAccounts.explain()

In [ ]:
dfAccounts.persist().count()

In [ ]:
# dfAccounts2= dfAccounts.where(col("FG_CURRENT")==1)
#.dropDuplicates(['ACCOUNTSERNO']).persist()

In [ ]:
dfAccounts2.persist().count()
#CON SEMIJOIN TARDÓ 17min, 8086359
#CON INNER JOIN + DROP TARDÓ  19 min 8086359
#FG_CURRENT 3266044

In [ ]:
dfAccounts2.select('SERNO').distinct().count()
#8086359

In [ ]:
dfAccounts2.where(col("FG_CURRENT")==1).count()

In [ ]:
dfAccounts2.where(col("FG_CURRENT")!=1).groupBy('stgeneral').count().show()

In [ ]:
#dfAccounts2.describe().toPandas()

In [ ]:
#LA LECTURA ES MUY LARGA, GUARDO LOS RESULTADOS PARA ACELERAR
dfAccounts2.write.parquet("/analytics/bc1_clustering/tables/dfAccountsRefdate")

In [ ]:
dfAccounts=dfAccounts2

In [ ]:
mostrarNulos(dfAccounts).show()

#### PEOPLE

In [30]:
#LECTURA DE TABLA ANALYTICS
dfPeople=spark.read.parquet("/analytics/bc1_clustering/tables/dfPeople")
dfPeople=dfPeople.withColumnRenamed("SERNO","PEOPLESERNO").withColumn('YOB',(dfPeople["DOB"]/10000).cast('integer'))

In [ ]:
#LECTURA DE TABLA PARQUET
dfPeople=spark.read.parquet("/user/sparta-server/trusted/tsys/PEOPLE")
# dfPeople.persist()

In [ ]:
dfPeople.printSchema()

In [ ]:
# dfPeople2=dfPeople.filter(dfPeople["FG_CURRENT"]==1)

In [ ]:
dfPeopleMax=dfPeople.groupBy('SERNO').agg(max('REFDATE').alias('FECHAMAX'))

dfPeople2=dfPeople.join(dfPeopleMax,(dfPeopleMax.SERNO==dfPeople.SERNO) & (dfPeopleMax.FECHAMAX==dfPeople.REFDATE),'leftsemi')

dfPeople2.printSchema()

In [ ]:
dfPeople2.persist().count()

In [ ]:
dfPeople2.select('SERNO').distinct().count()

In [ ]:
dfPeople2=dfPeople2.dropDuplicates(['SERNO'])

In [ ]:
dfPeople2.count()

In [ ]:
#ESCRITURA DE LA TABLA
dfPeople2.write.parquet("/analytics/bc1_clustering/tables/dfPeople")

In [ ]:
dfCupom.join(dfPeople,dfCupom.PES_NU_CPFCNPJ==dfPeople.SSNUMBER).count()

In [ ]:
dfPeople.describe().show()

In [ ]:
mostrarNulos(dfPeople).show()

#### TRANSACTIONS TSYS

In [56]:
#MEJOR LEER LA TABLA PROCESADA!
dfTrantsysTratada=spark.read.parquet("/analytics/bc1_clustering/tables/dfTrantsysTratada02Ene")
# dfTrantsysTratada=spark.read.parquet("/analytics/bc1_clustering/tables/dfTrantsysTratada11Dic")

In [9]:
#LECTURA DE TABLA PARQUET
#Landing
# dfTrantsys=spark.read.parquet("/user/sparta-server/landing/tsys/CTRANSACTIONS/DT_LANDING=20171202")
#Trusted
dfTrantsys=spark.read.parquet("/user/sparta-server/trusted/tsys/CTRANSACTIONS")
dfTrantsys=dfTrantsys.withColumnRenamed("SERNO","TRANSACTIONSERNO") \
    .withColumnRenamed("CACCSERNO","ACCOUNTSERNOFK")

In [10]:
dfTrantsys.printSchema()

root
 |-- TYPE: string (nullable = true)
 |-- INSTITUTION_ID: integer (nullable = true)
 |-- TRANSACTIONSERNO: long (nullable = true)
 |-- PARTITIONKEY: long (nullable = true)
 |-- ACCOUNTSERNOFK: long (nullable = true)
 |-- CARDSERNO: long (nullable = true)
 |-- PRODUCT: integer (nullable = true)
 |-- BATCHSERNO: long (nullable = true)
 |-- TYPESERNO_ALLOC: long (nullable = true)
 |-- TYPESERNO_FEES: long (nullable = true)
 |-- TYPESERNO_REPORTS: long (nullable = true)
 |-- TYPESERNO_REWARDS: long (nullable = true)
 |-- TYPESERNO_GLEDGER: long (nullable = true)
 |-- TYPESERNO_DIVERT: long (nullable = true)
 |-- ORIG_MSG_TYPE: string (nullable = true)
 |-- I000_MSG_TYPE: string (nullable = true)
 |-- I002_NUMBER: string (nullable = true)
 |-- I003_PROC_CODE: string (nullable = true)
 |-- I004_AMT_TRXN: double (nullable = true)
 |-- I005_AMT_SETTLE: double (nullable = true)
 |-- I006_AMT_BILL: double (nullable = true)
 |-- I007_LOAD_DATE: string (nullable = true)
 |-- I008_BILLING_FEE: 

In [ ]:
# dfTrantsys= dfTrantsys.select("_c2","_c4","_c12").withColumnRenamed("_c2","TRANSACTIONSERNO") \
#     .withColumnRenamed("_c4","ACCOUNTSERNOFK").withColumnRenamed("_c12","TYPESERNO_GLEDGER") 

In [ ]:
dfTrantsys.count()

In [ ]:
# dfTrantsys2=dfTrantsys.where(col('FG_CURRENT')==1)

In [11]:
dfTrantsysMax=dfTrantsys.groupBy('TRANSACTIONSERNO').agg(max('REFDATE').alias('FECHAMAX'))

dfTrantsys2=dfTrantsys.join(dfTrantsysMax,(dfTrantsysMax.TRANSACTIONSERNO==dfTrantsys.TRANSACTIONSERNO) & (dfTrantsysMax.FECHAMAX==dfTrantsys.REFDATE),'leftsemi')

In [ ]:
dfTrantsys2.count()

In [ ]:
dfTrantsys2.select('TRANSACTIONSERNO').distinct().count()

In [12]:
dfTrantsys3=dfTrantsys2.dropDuplicates(['TRANSACTIONSERNO'])

In [13]:
#CARGAR dfCodeproducts ANTES DE ESTO
BCdfCodeproducts=broadcast(dfCodeproducts.select("SERNO","PRODUTO"))

In [14]:
dfTrantsysTratada=dfTrantsys3.join(BCdfCodeproducts,BCdfCodeproducts.SERNO==dfTrantsys.TYPESERNO_GLEDGER) \
    .select('TRANSACTIONSERNO','ACCOUNTSERNOFK','TYPESERNO_GLEDGER','I013_TRXN_DATE','I018_MERCH_TYPE','AMOUNT','VALUEDATE','POSTDATE','STGENERAL','REFDATE','FG_CURRENT','TM_INSERT','PRODUTO')

In [15]:
dfTrantsysTratada.printSchema()

root
 |-- TRANSACTIONSERNO: long (nullable = true)
 |-- ACCOUNTSERNOFK: long (nullable = true)
 |-- TYPESERNO_GLEDGER: long (nullable = true)
 |-- I013_TRXN_DATE: string (nullable = true)
 |-- I018_MERCH_TYPE: string (nullable = true)
 |-- AMOUNT: double (nullable = true)
 |-- VALUEDATE: string (nullable = true)
 |-- POSTDATE: string (nullable = true)
 |-- STGENERAL: string (nullable = true)
 |-- REFDATE: timestamp (nullable = true)
 |-- FG_CURRENT: integer (nullable = true)
 |-- TM_INSERT: timestamp (nullable = true)
 |-- PRODUTO: string (nullable = true)



In [16]:
#SOLO LAS TRANSACCIONES NECESARIAS
dfTrantsysTratada.write.parquet("/analytics/bc1_clustering/tables/dfTrantsysTratada02Ene")

In [17]:
dfTrantsysTratada=spark.read.parquet("/analytics/bc1_clustering/tables/dfTrantsysTratada02Ene")

In [ ]:
dfTrantsysTratada.count()

In [ ]:
dfTrantsysTratada.where(col('POSTDATE')>='20151201').count()

In [ ]:
dfTrantsysTratada.where(col('I013_TRXN_DATE')>='20151201').count()

In [ ]:
dfTrantsysTratada.where(col('REFDATE')>='20151201').count()

In [ ]:
dfTrantsys.count()

In [ ]:
dfTrantsys2.select('SERNO').distinct().count()

In [ ]:
dfTrantsys2.describe().show()

In [ ]:
mostrarNulos(dfTrantsys2).show()

In [ ]:
dfTrantsys.agg({'I013_TRXN_DATE':'min','I007_LOAD_DATE':'min','I015_SETTLE_DATE':'min','VALUEDATE':'min','POSTDATE':'min'}).show()

In [ ]:
mostrarNulos(dfTrantsys.select('I007_LOAD_DATE','I013_TRXN_DATE','I015_SETTLE_DATE','REFDATE','VALUEDATE','POSTDATE')).show()

#### CEXTENSION

In [57]:
#LECTURA DE TABLA PARQUET
dfExtension=spark.read.parquet("/user/sparta-server/trusted/tsys/CEXTENSION")

In [ ]:
dfExtension.printSchema()

In [ ]:
dfExtension.count()

In [ ]:
# dfExtension2Max=dfExtension2.groupBy('ROW_SERNO').agg(max('REFDATE').alias('FECHAMAX'))

# dfExtension3=dfExtension2.join(dfExtension2Max,(dfExtension2Max.ROW_SERNO==dfExtension2.ROW_SERNO) & (dfExtension2Max.FECHAMAX==dfExtension2.REFDATE),'leftsemi')

# dfExtension3.printSchema()

In [ ]:
# dfExtension3.count()
#2645496

In [ ]:
dfExtension.describe().show()

In [ ]:
mostrarNulos(dfTrantsys.select('')).show()

#### STATEMENTS

In [58]:
#LECTURA DE TABLA PARQUET
dfStatements=spark.read.parquet("/user/sparta-server/trusted/tsys/CSTATEMENTS")

In [ ]:
dfStatements.printSchema()

In [ ]:
dfStatements.persist().count()

In [ ]:
dfStatements.where(col('FG_CURRENT')==1).count()

In [ ]:
# dfStatements2=dfStatements.where(col('FG_CURRENT')==1).persist()

In [ ]:
# dfStatementsMax=dfStatements.groupBy('SERNO').agg(max('REFDATE').alias('FECHAMAX'))

# dfStatements2=dfStatements.join(dfStatementsMax,(dfStatementsMax.SERNO==dfStatements.SERNO) & (dfStatementsMax.FECHAMAX==dfStatements.REFDATE),'leftsemi')

# dfStatements2.printSchema()

In [ ]:
# dfStatements2.count()
# 60592044

In [ ]:
dfStatements.select('SERNO').distinct().count()

In [ ]:
dfStatements.describe().show()

In [ ]:
mostrarNulos(dfStatements).show()

#### STATEMENTINSTOFFERS

In [59]:
#LECTURA DE TABLA PARQUET
dfStatementinstoffers=spark.read.parquet("/user/sparta-server/trusted/tsys/CSTATEMENTINSTOFFERS")

In [ ]:
dfStatementinstoffers.printSchema()

In [ ]:
dfStatementinstoffers.persist().count()

In [ ]:
dfStatementinstoffers=dfStatementinstoffers.where(col('FG_CURRENT')==1).persist()

In [ ]:
# dfStatementinstoffersMax=dfStatementinstoffers.groupBy('SERNO').agg(max('REFDATE').alias('FECHAMAX'))

# dfStatementinstoffers2=dfStatementinstoffers.join(dfStatementinstoffersMax,(dfStatementinstoffersMax.SERNO==dfStatementinstoffers.SERNO) & (dfStatementinstoffersMax.FECHAMAX==dfStatementinstoffers.REFDATE),'leftsemi')

# dfStatementinstoffers2.printSchema()

In [ ]:
# dfStatementinstoffers2.count()
#398251162

In [ ]:
dfStatementinstoffers.select('SERNO').distinct().count()

In [ ]:
dfStatementinstoffers2.describe().show()

In [ ]:
mostrarNulos(dfStatementinstoffers2).show()

#### RISKDOMAINS

In [60]:
#LECTURA DE TABLA PARQUET
dfRiskdomains=spark.read.parquet("/user/sparta-server/trusted/tsys/RISKDOMAINS")

In [ ]:
dfRiskdomains.printSchema()

In [ ]:
dfRiskdomains.persist().count()

In [ ]:
# dfRiskdomainsMax=dfRiskdomains.groupBy('SERNO').agg(max('REFDATE').alias('FECHAMAX'))

# dfRiskdomains2=dfRiskdomains.join(dfRiskdomainsMax,(dfRiskdomainsMax.SERNO==dfRiskdomains.SERNO) & (dfRiskdomainsMax.FECHAMAX==dfRiskdomains.REFDATE),'leftsemi')

# dfRiskdomains2.printSchema()

In [ ]:
dfRiskdomains2.describe().show()

In [ ]:
mostrarNulos(dfRiskdomains2).show()

# Predictive Tables

### Estrutura Mercadologica

In [10]:
#CARGAR LA EM (no hace faltar ejecutar lo siguiente)
dfEM=spark.read.parquet("/analytics/bc1_clustering/tables/dfEM")

dfEM.persist()

dfEM.show(5)

+-----------+-----------------+--------------------+-----------------+-----------------+--------------+--------------+------------+------------+------------+-------------------+-------------------+-------------------+
|IVA_CD_ITEM|FOR_CD_FORNECEDOR|         IVA_DS_ITEM|PRF_CD_SUBFAMILIA|PRF_DS_SUBFAMILIA|PRF_CD_FAMILIA|PRF_DS_FAMILIA|PRF_CD_GRUPO|PRF_DS_GRUPO|PRF_CD_SETOR|       PRF_DS_SETOR|PRF_CD_DEPARTAMENTO|PRF_DS_DEPARTAMENTO|
+-----------+-----------------+--------------------+-----------------+-----------------+--------------+--------------+------------+------------+------------+-------------------+-------------------+-------------------+
|    2999641|           290009|PRODUTO GENERICO ...|       5066990100|         DIVERSOS|      50669901|      DIVERSOS|      506699|       GENCO|        5066|TEXTIL - PERMANENTE|                  5|             TEXTIL|
|    2999633|           290009|PRODUTO GENERICO ...|       5066990100|         DIVERSOS|      50669901|      DIVERSOS|      5066

In [11]:
dfEMtmp = dfItemvarejo.join(dfProdutofam, dfItemvarejo.PRF_CD_SUBFAMILIA==dfProdutofam.PRF_CD_SUBFAMILIA,'left_outer') \
    .join(dfProdutofamfam, dfProdutofamfam.PRF_CD_FAMILIA==dfProdutofam.PRF_CD_FAMILIA,'left_outer') \
    .join(dfProdutofamgrp, dfProdutofamgrp.PRF_CD_GRUPO==dfProdutofamfam.PRF_CD_GRUPO,'left_outer') \
    .join(dfProdutofamset, dfProdutofamset.PRF_CD_SETOR==dfProdutofamgrp.PRF_CD_SETOR,'left_outer') \
    .join(dfProdutofamdep, dfProdutofamdep.PRF_CD_DEPARTAMENTO==dfProdutofamset.PRF_CD_DEPARTAMENTO,'left_outer') 
dfEMtmp.printSchema()

NameError: name 'dfItemvarejo' is not defined

In [37]:
dfEM=dfEMtmp.select("IVA_CD_ITEM","FOR_CD_FORNECEDOR","IVA_DS_ITEM",dfProdutofam["PRF_CD_SUBFAMILIA"],"PRF_DS_SUBFAMILIA", \
              dfProdutofamfam["PRF_CD_FAMILIA"],"PRF_DS_FAMILIA",dfProdutofamgrp["PRF_CD_GRUPO"],"PRF_DS_GRUPO", \
              dfProdutofamset["PRF_CD_SETOR"],"PRF_DS_SETOR",dfProdutofamdep["PRF_CD_DEPARTAMENTO"],"PRF_DS_DEPARTAMENTO").persist()

In [38]:
dfEM.printSchema()

root
 |-- IVA_CD_ITEM: long (nullable = true)
 |-- FOR_CD_FORNECEDOR: long (nullable = true)
 |-- IVA_DS_ITEM: string (nullable = true)
 |-- PRF_CD_SUBFAMILIA: long (nullable = true)
 |-- PRF_DS_SUBFAMILIA: string (nullable = true)
 |-- PRF_CD_FAMILIA: long (nullable = true)
 |-- PRF_DS_FAMILIA: string (nullable = true)
 |-- PRF_CD_GRUPO: long (nullable = true)
 |-- PRF_DS_GRUPO: string (nullable = true)
 |-- PRF_CD_SETOR: integer (nullable = true)
 |-- PRF_DS_SETOR: string (nullable = true)
 |-- PRF_CD_DEPARTAMENTO: integer (nullable = true)
 |-- PRF_DS_DEPARTAMENTO: string (nullable = true)



In [39]:
mostrarNulos(dfEM).toPandas()

,IVA_CD_ITEM,FOR_CD_FORNECEDOR,IVA_DS_ITEM,PRF_CD_SUBFAMILIA,PRF_DS_SUBFAMILIA,PRF_CD_FAMILIA,PRF_DS_FAMILIA,PRF_CD_GRUPO,PRF_DS_GRUPO,PRF_CD_SETOR,PRF_DS_SETOR,PRF_CD_DEPARTAMENTO,PRF_DS_DEPARTAMENTO
0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [40]:
dfEM.is_cached

True

In [41]:
dfEM.count()

456658

In [ ]:
dfEM.write.parquet("/analytics/bc1_clustering/tables/dfEM")

In [12]:
dfEM.select('PRF_CD_SUBFAMILIA').distinct().count()

3017

### User - Purchases

In [ ]:
#PARA EVITAR EL TIMEOUT DE BROADCAST
#spark.conf.set("spark.sql.broadcastTimeout",300)


In [23]:
dfCupom.where((to_date(col('CUP_DT_CUPOM'))>='2016-11-01') & (to_date(col('CUP_DT_CUPOM'))<'2017-11-01')) \
    .select('CUP_DT_CUPOM',to_date(col('CUP_DT_CUPOM')).alias('FECHA')).agg({"FECHA": "min", "CUP_DT_CUPOM": "max"}).show()

+----------+--------------------+
|min(FECHA)|   max(CUP_DT_CUPOM)|
+----------+--------------------+
|2016-11-01|2017-10-31 00:00:...|
+----------+--------------------+



In [24]:
dfCupomFilter=dfCupom.where((to_date(col('CUP_DT_CUPOM'))>='2016-11-01') & (to_date(col('CUP_DT_CUPOM'))<'2017-11-01')) \
    .select('CUP_ID_CUPOM','PES_NU_CPFCNPJ','TRA_CD_TRANSACAO')

In [25]:
#dfCupomFilter.persist().count()

In [26]:
dfTransacaoFilter=dfTransacao.where((to_date(col('TRA_DT_TRANSACAO'))>='2016-11-01') & (to_date(col('TRA_DT_TRANSACAO'))<'2017-11-01')) \
    .select('TRA_CD_TRANSACAO','CTA_CD_CONTA')

In [ ]:
#dfTransacaoFilter.persist().count()

In [ ]:
# dfTransacaoFilter.show(5)

In [27]:
dfCupomFilter.printSchema()

root
 |-- CUP_ID_CUPOM: long (nullable = true)
 |-- PES_NU_CPFCNPJ: long (nullable = true)
 |-- TRA_CD_TRANSACAO: long (nullable = true)



In [31]:
dfPeopleFilter=dfPeople.select('PEOPLESERNO','SSNUMBER').withColumn('SSNUMBER',dfPeople["SSNUMBER"].cast('long')).persist()
dfPeopleFilter.count()

13945016

In [32]:
dfPeopleFilter.printSchema()

root
 |-- PEOPLESERNO: long (nullable = true)
 |-- SSNUMBER: long (nullable = true)



In [35]:
dfAccountsFilter=dfAccounts.select('ACCOUNTSERNO','PEOPLESERNOFK')
#dfAccountsFilter.persist().count()

In [36]:
# #EJECUCIÓN POR PASOS, SI NO FUNCIONA LA COMPLETA

# dftest1=dfCupomFilter.join(dfTransacaoFilter,dfCupomFilter.TRA_CD_TRANSACAO==dfTransacaoFilter.TRA_CD_TRANSACAO,"left_outer") \
#     .drop(dfCupomFilter.TRA_CD_TRANSACAO).drop(dfTransacaoFilter.TRA_CD_TRANSACAO)
# # dftest1.persist()
# # dftest1.count()

# dftest1.printSchema()

# dftest1.printSchema()

# dftest1.write.parquet("/analytics/bc1_clustering/tables/dftest1")

# dftest1=spark.read.parquet("/analytics/bc1_clustering/tables/dftest1")

# dftest2=dftest1.join(dfPeopleFilter,dftest1.PES_NU_CPFCNPJ==dfPeopleFilter.SSNUMBER,"left_outer")
# # dftest2.persist()
# # dftest2.count()

# dftest2.write.parquet("/analytics/bc1_clustering/tables/dftest2")

# dftest1.unpersist()

# dfPeopleFilter.unpersist()

# dfAccountsFilter.printSchema()

# dftest3=dftest2.join(dfAccountsFilter,dftest2.PEOPLESERNO==dfAccountsFilter.PEOPLESERNOFK,"left_outer")
# # dftest3.persist()
# # dftest3.count()

# dftest3.write.parquet("/analytics/bc1_clustering/tables/dftest3")

# dftest2.unpersist()

# new_column_1 = expr(
#     """IF(CTA_CD_CONTA IS NULL, ACCOUNTSERNO, CTA_CD_CONTA)"""
# )

# dfTemp=dftest3.filter(col("CTA_CD_CONTA").isNotNull() | col("ACCOUNTSERNO").isNotNull()) \
#     .withColumn("ACCOUNTPK",new_column_1)
# # dfTemp.persist()
# # dfTemp.count()

# dftest3.unpersist()

In [37]:
#CON ACCOUNT Y MONTO DE DINERO POR PRODUCTO
new_column_1 = expr(
    """IF(CTA_CD_CONTA IS NULL, ACCOUNTSERNO, CTA_CD_CONTA)"""
)

dfTemp= dfCupomFilter.join(dfTransacaoFilter,dfCupomFilter.TRA_CD_TRANSACAO==dfTransacaoFilter.TRA_CD_TRANSACAO,"left_outer") \
    .drop(dfCupomFilter.TRA_CD_TRANSACAO).drop(dfTransacaoFilter.TRA_CD_TRANSACAO) \
    .join(dfPeopleFilter,dfCupomFilter.PES_NU_CPFCNPJ==dfPeopleFilter.SSNUMBER,"left_outer") \
    .join(dfAccountsFilter,dfPeopleFilter.PEOPLESERNO==dfAccountsFilter.PEOPLESERNOFK,"left_outer") \
    .filter(col("CTA_CD_CONTA").isNotNull() | col("ACCOUNTSERNO").isNotNull()) \
    .withColumn("ACCOUNTPK",new_column_1) #.persist()

In [ ]:
#CON ACCOUNT Y MONTO DE DINERO POR PRODUCTO
# new_column_1 = expr(
#     """IF(CTA_CD_CONTA IS NULL, ACCOUNTSERNO, CTA_CD_CONTA)"""
# )

# dfTemp= dfCupom.join(dfTransacao,dfCupom.TRA_CD_TRANSACAO==dfTransacao.TRA_CD_TRANSACAO,"left_outer") \
#     .join(dfPeople,dfCupom.PES_NU_CPFCNPJ==dfPeople.SSNUMBER,"left_outer") \
#     .join(dfAccounts,dfPeople.PEOPLESERNO==dfAccounts.PEOPLESERNOFK,"left_outer") \
#     .filter(col("CTA_CD_CONTA").isNotNull() | col("ACCOUNTSERNO").isNotNull()) \
#     .withColumn("ACCOUNTPK",new_column_1) #.persist()

In [ ]:
dfTemp.persist()

In [ ]:
dfTemp.count()
# dfCupom.unpersist() 29032591
# dfTransacao.unpersist()

In [ ]:
dfTemp.groupBy("CUP_ID_CUPOM","ACCOUNTPK").count().where(col('count')>1).show(5)

In [ ]:
dfTemp.groupBy("CUP_ID_CUPOM","ACCOUNTPK").count().groupBy('count').count().show(50)

In [38]:
#CREACIÓN DEL DATAFRAME REDUCIDO
dfTempRed=dfTemp.select("CUP_ID_CUPOM","ACCOUNTPK").distinct()

In [ ]:
dfTempRed.groupBy("CUP_ID_CUPOM").count().where(col('count')>1).count()
#dfTempRed.groupBy("CUP_ID_CUPOM").count().where(col('count')>1).show(5)

In [ ]:
# dfTemp2.unpersist()

In [ ]:
dfTempRed.filter(col("CUP_ID_CUPOM")==1496147340).show(5)

In [ ]:
dfTempRed.select("CUP_ID_CUPOM").distinct().count()
#28586691

In [ ]:
dfTempRed.write.parquet("/analytics/bc1_clustering/tables/dfTempRedVal19Dic")

In [61]:
dfTempRed=spark.read.parquet("/analytics/bc1_clustering/tables/dfTempRedVal19Dic")

In [ ]:
# #CHEQUEO
# # ACCOUNT 11030376 GROUP 101054 MONEY 150.64
# dfTest=dfTempRed.filter(col("ACCOUNTPK")==11030376)

# dfTest2=dfItemcupom.join(broadcast(dfTest),dfItemcupom.CUP_ID_CUPOM==dfTest.CUP_ID_CUPOM).persist()

# dfTest3=dfTest2.join(broadcast(dfEM),dfTest2.IVA_CD_ITEM==dfEM.IVA_CD_ITEM).filter(col("PRF_CD_GRUPO")==101054).persist()

# dfTest3.agg(sum("ICU_VL_ITENS")).show(5)

In [39]:
#DESDE TempRed

dfTemp2=dfTempRed.join(dfItemcupom,dfTempRed.CUP_ID_CUPOM==dfItemcupom.CUP_ID_CUPOM).groupBy("ACCOUNTPK","IVA_CD_ITEM") \
    .agg(sum('ICU_VL_ITENS').alias('PURCHASES')).join(broadcast(dfEM),dfItemcupom.IVA_CD_ITEM==dfEM.IVA_CD_ITEM)


#DESDE Temp
# dfTemp2=dfTemp.join(dfItemcupom,dfTemp.CUP_ID_CUPOM==dfItemcupom.CUP_ID_CUPOM).groupBy("ACCOUNTPK","IVA_CD_ITEM") \
#     .agg(sum('ICU_VL_ITENS').alias('PURCHASES')).join(dfEM,dfItemcupom.IVA_CD_ITEM==dfEM.IVA_CD_ITEM).persist()

In [ ]:
dfTemp2.printSchema()

In [ ]:
dfTemp.show(5)

In [ ]:
dfTemp2.show(5)

In [ ]:
#CON LISTA DE CATEGORIAS ENCONTRADAS
dfBuffer=dfTemp2.groupBy("ACCOUNTPK").pivot("PRF_CD_SUBFAMILIA").sum("PURCHASES")

#CON LISTA COMPLETA DE EST. MERCADOLOGICA
#dfBuffer=dfTemp2.groupBy("ACCOUNTPK").pivot("PRF_CD_GRUPO",groupList).sum("PURCHASES").persist()

In [ ]:
#dfBuffer.unpersist()
#dfTemp2.filter(dfTemp2['ACCOUNTPK']==3796007).groupBy('ACCOUNTPK','PRF_CD_GRUPO').sum('PURCHASES').show()

In [ ]:
len(dfBuffer.columns)

In [ ]:
dfBuffer.show(5)

### User - Financial Product

In [ ]:
#CARGAR dfTrantsysTratada más abajo

In [66]:
BCdfCodeproducts=broadcast(dfCodeproducts.select("SERNO","PRODUTO"))

In [67]:
dfTempTranTsys=dfTrantsysTratada.where((col('I013_TRXN_DATE')>='20151101') & (col('I013_TRXN_DATE')<'20171101')) \
    .groupBy("ACCOUNTSERNOFK","PRODUTO").agg(count('*').alias('TRANSACTIONS'))

In [68]:
# dfTempTranTsys=dfTrantsys.join(BCdfCodeproducts,BCdfCodeproducts.SERNO==dfTrantsys.TYPESERNO_GLEDGER) \
#     .groupBy("ACCOUNTSERNOFK","PRODUTO").agg(count('SERNO').alias('TRANSACTIONS'))

In [69]:
dfTempTranTsys.printSchema()

root
 |-- ACCOUNTSERNOFK: long (nullable = true)
 |-- PRODUTO: string (nullable = true)
 |-- TRANSACTIONS: long (nullable = false)



In [ ]:
# dfTempTranTsys.write.parquet("/analytics/bc1_clustering/tables/dfTempTranTsys")

In [ ]:
# dfTempTranTsys=spark.read.parquet("/analytics/bc1_clustering/tables/dfTempTranTsys")

In [ ]:
# #CON ACCOUNT
# dfTempTsys=dfTempRed.select("ACCOUNTPK").distinct() \
#     .join(dfTempTranTsys,dfTempTranTsys.ACCOUNTSERNOFK==dfTempRed.ACCOUNTPK) 

In [ ]:
# dfTsys=dfTempTsys.groupBy("ACCOUNTPK").pivot("PRODUTO").sum("TRANSACTIONS").fillna(0)

In [70]:
dfTsys=dfTempTranTsys.groupBy("ACCOUNTSERNOFK").pivot("PRODUTO").sum("TRANSACTIONS").fillna(0)

In [71]:
dfTsys.count()

3156089

In [72]:
dfTsys.filter(col("Emprestimo_Pessoal")>0).count()

29019

In [73]:
dfTsys.createOrReplaceTempView("tTsys")

In [74]:
dfTsysBin=spark.sql("""SELECT ACCOUNTSERNOFK as ACCOUNTPK, 
    SUM(CASE WHEN Emprestimo_Pessoal>0 THEN 1 ELSE 0 END) Emprestimo_Pessoal,
    SUM(CASE WHEN Pagamento_de_Contas>0 THEN 1 ELSE 0 END) Pagamento_de_Contas, 
    SUM(CASE WHEN Parcelamento_Fatura_Automatico>0 THEN 1 ELSE 0 END) Parcelamento_Fatura_Automatico, 
    SUM(CASE WHEN Parcelamento_Fatura_Facil>0 THEN 1 ELSE 0 END) Parcelamento_Fatura_Facil, 
    SUM(CASE WHEN Saque_Parcelado>0 THEN 1 ELSE 0 END) Saque_Parcelado, 
    SUM(CASE WHEN Saque_a_Vista>0 THEN 1 ELSE 0 END) Saque_a_Vista,
    SUM(CASE WHEN Overlimit>0 THEN 1 ELSE 0 END) Overlimit
    from tTsys group by ACCOUNTSERNOFK """)

In [75]:
dfTsysBin.printSchema()

root
 |-- ACCOUNTPK: long (nullable = true)
 |-- Emprestimo_Pessoal: long (nullable = true)
 |-- Pagamento_de_Contas: long (nullable = true)
 |-- Parcelamento_Fatura_Automatico: long (nullable = true)
 |-- Parcelamento_Fatura_Facil: long (nullable = true)
 |-- Saque_Parcelado: long (nullable = true)
 |-- Saque_a_Vista: long (nullable = true)
 |-- Overlimit: long (nullable = true)



In [ ]:
dfTsysBin.describe().toPandas()

### Off-us

In [ ]:
# dfProdOffUs=broadcast(dfCodeproducts.select("SERNO","PRODUTO").where(col('PRODUTO')=='Off_Us'))

In [ ]:
# dfProdOffUs.printSchema()

In [76]:
dfOffUs=dfTrantsysTratada.where(col('PRODUTO')=='Off_Us') \
    .where((col('I013_TRXN_DATE')>='20161101') & (col('I013_TRXN_DATE')<'20171101')) \
    .where(col('STGENERAL')!='REJC') \
    .select('ACCOUNTSERNOFK','I018_MERCH_TYPE','AMOUNT') \
    .groupBy("ACCOUNTSERNOFK").pivot("I018_MERCH_TYPE").sum("AMOUNT").fillna(0)

In [77]:
# dfOffUs=dfTrantsys.join(dfProdOffUs,dfProdOffUs.SERNO==dfTrantsys.TYPESERNO_GLEDGER).where(col('STGENERAL')!='REJC') \
#     .select('ACCOUNTSERNOFK','I018_MERCH_TYPE','AMOUNT') \
#     .groupBy("ACCOUNTSERNOFK").pivot("I018_MERCH_TYPE").sum("AMOUNT").fillna(0)

In [78]:
dfOffUs.printSchema()

root
 |-- ACCOUNTSERNOFK: long (nullable = true)
 |-- : double (nullable = false)
 |-- 0742: double (nullable = false)
 |-- 0763: double (nullable = false)
 |-- 0780: double (nullable = false)
 |-- 1520: double (nullable = false)
 |-- 1711: double (nullable = false)
 |-- 1731: double (nullable = false)
 |-- 1740: double (nullable = false)
 |-- 1750: double (nullable = false)
 |-- 1761: double (nullable = false)
 |-- 1771: double (nullable = false)
 |-- 1799: double (nullable = false)
 |-- 2741: double (nullable = false)
 |-- 2791: double (nullable = false)
 |-- 2842: double (nullable = false)
 |-- 3000: double (nullable = false)
 |-- 3001: double (nullable = false)
 |-- 3005: double (nullable = false)
 |-- 3007: double (nullable = false)
 |-- 3008: double (nullable = false)
 |-- 3009: double (nullable = false)
 |-- 3010: double (nullable = false)
 |-- 3011: double (nullable = false)
 |-- 3012: double (nullable = false)
 |-- 3013: double (nullable = false)
 |-- 3015: double (nullable = 

### Definitive accounts

In [79]:
dfUniqueAcc=dfBuffer.select('ACCOUNTPK').union(dfOffUs.select('ACCOUNTSERNOFK')).distinct()

In [80]:
dfUniqueAcc.printSchema()

root
 |-- ACCOUNTPK: long (nullable = true)



In [81]:
dfUniqueAcc.count()

3991865

In [82]:
dfUniqueAcc.select('ACCOUNTPK').distinct().count()

3991865

In [83]:
dfBuffer.agg(countDistinct('ACCOUNTPK'),count('*')).show()
#2648132| 2648132

+-------------------------+--------+
|count(DISTINCT ACCOUNTPK)|count(1)|
+-------------------------+--------+
|                  3230266| 3230266|
+-------------------------+--------+



In [84]:
dfOffUs.agg(countDistinct('ACCOUNTSERNOFK'),count('*')).show()
#3024473| 3024473

+------------------------------+--------+
|count(DISTINCT ACCOUNTSERNOFK)|count(1)|
+------------------------------+--------+
|                       2477678| 2477678|
+------------------------------+--------+



In [85]:
dfDefAcc=dfAccounts.join(dfUniqueAcc,dfUniqueAcc.ACCOUNTPK==dfAccounts.ACCOUNTSERNO) \
    .select(dfUniqueAcc.ACCOUNTPK,dfAccounts.CUSTSERNO,dfAccounts.BALANCE,dfAccounts.INSTALMENTBALANCEFOROVLMCHECK,dfAccounts.CREDITLIMIT )

In [86]:
dfDefAcc.printSchema()

root
 |-- ACCOUNTPK: long (nullable = true)
 |-- CUSTSERNO: long (nullable = true)
 |-- BALANCE: double (nullable = true)
 |-- INSTALMENTBALANCEFOROVLMCHECK: double (nullable = true)
 |-- CREDITLIMIT: double (nullable = true)



In [87]:
dfDefAcc.persist().count()

3991780

In [88]:
dfDefAcc.select('ACCOUNTPK').distinct().count()

3991780

### Overlimit Authorization

In [89]:
dfExtension2=dfExtension.where(to_date(col('REFDATE'))<'2017-11-01') \
    .where((col('ENTITY_INDICATOR')=='A') & (col('FIELD_NO')==1532))

dfExtension2Max=dfExtension2.groupBy('ROW_SERNO').agg(max('REFDATE').alias('FECHAMAX'))

dfExtension3=dfExtension2.join(dfExtension2Max,(dfExtension2Max.ROW_SERNO==dfExtension2.ROW_SERNO) & (dfExtension2Max.FECHAMAX==dfExtension2.REFDATE),'leftsemi')


In [90]:
dfOverlimitAuth=dfExtension3.groupBy('ROW_SERNO').agg(max('VALUE').alias('Overlimit_Auth')) \
    .withColumn('Overlimit_Auth',when(col('Overlimit_Auth')=='1',1).otherwise(0))
#     .withColumn('Overlimit2',when(col('Overlimit')=='1',1).otherwise(0))

In [91]:
dfOverlimitAuth.printSchema()

root
 |-- ROW_SERNO: long (nullable = true)
 |-- Overlimit_Auth: integer (nullable = false)



In [92]:
dfOverlimitAuth.persist()

DataFrame[ROW_SERNO: bigint, Overlimit_Auth: int]

In [59]:
dfOverlimitAuth.describe().show()

+-------+--------------------+------------------+
|summary|           ROW_SERNO|    Overlimit_Auth|
+-------+--------------------+------------------+
|  count|             2647548|           2647548|
|   mean|1.0732928123612868E7|0.5972144792086866|
| stddev|   3572317.003341495|0.4904583936376508|
|    min|                  18|                 0|
|    max|            13429722|                 1|
+-------+--------------------+------------------+



In [93]:
dfOverlimitAuth.groupBy('Overlimit_Auth').count().show()

+--------------+-------+
|Overlimit_Auth|  count|
+--------------+-------+
|             1|1581154|
|             0|1066394|
+--------------+-------+



### Closing balance & Installments offers

In [94]:
dfStatementsMax=dfStatements.where(to_date(col('REFDATE'))<'2017-11-01') \
    .groupBy('ACC_SERNO').agg(max('REFDATE').alias('FECHAMAX'))

dfStatements2=dfStatements.join(dfStatementsMax,(dfStatementsMax.ACC_SERNO==dfStatements.ACC_SERNO) & (dfStatementsMax.FECHAMAX==dfStatements.REFDATE),'leftsemi')

# dfStatements2.printSchema()

In [95]:
dfStatementinstoffersMax=dfStatementinstoffers.groupBy('SERNO').agg(max('REFDATE').alias('FECHAMAX'))

dfStatementinstoffers2=dfStatementinstoffers.join(dfStatementinstoffersMax,(dfStatementinstoffersMax.SERNO==dfStatementinstoffers.SERNO) & (dfStatementinstoffersMax.FECHAMAX==dfStatementinstoffers.REFDATE),'leftsemi')

# dfStatementinstoffers2.printSchema()

In [96]:
dfClBalInstOff=dfStatements2.join(dfStatementinstoffers2, dfStatements2.SERNO==dfStatementinstoffers2.STMTSERNO, 'left_outer') \
    .groupBy('ACC_SERNO','CLOSING_BALANCE').agg(countDistinct(dfStatementinstoffers2['SERNO']).alias('INSTALLMENTSOFFERS'))
#     .withColumnRenamed('count','INSTALLMENTSOFFERS')

In [97]:
dfClBalInstOff.printSchema()

root
 |-- ACC_SERNO: long (nullable = true)
 |-- CLOSING_BALANCE: double (nullable = true)
 |-- INSTALLMENTSOFFERS: long (nullable = false)



In [98]:
dfClBalInstOff.persist().count()

4105426

In [99]:
dfClBalInstOff.describe('INSTALLMENTSOFFERS').show()

+-------+------------------+
|summary|INSTALLMENTSOFFERS|
+-------+------------------+
|  count|           4105426|
|   mean| 5.233881697051658|
| stddev| 4.130780233632609|
|    min|                 0|
|    max|                28|
+-------+------------------+



### Risk Domains

In [100]:
dfRiskdomains.printSchema()

root
 |-- TYPE: string (nullable = true)
 |-- INSTITUTION_ID: integer (nullable = true)
 |-- SERNO: long (nullable = true)
 |-- TABINDICATOR: string (nullable = true)
 |-- CUSTSERNO: long (nullable = true)
 |-- HIERARCHYSERNO: long (nullable = true)
 |-- NUMBERX: string (nullable = true)
 |-- NAME: string (nullable = true)
 |-- MAKEAUTHGROUP: integer (nullable = true)
 |-- AUTHCURRENCY: string (nullable = true)
 |-- AUTHLIMIT: double (nullable = true)
 |-- AUTHLIMITPERIOD: integer (nullable = true)
 |-- AUTHALLTRXN: integer (nullable = true)
 |-- AUTHALLTRXNPERIOD: integer (nullable = true)
 |-- AUTHUSAGECODE: long (nullable = true)
 |-- AUTHUDECLINETRXN: integer (nullable = true)
 |-- AUTHUDECLINELIMIT: double (nullable = true)
 |-- AUTHUDECLINEPERIOD: integer (nullable = true)
 |-- AUTHCASHLIMIT: double (nullable = true)
 |-- AUTHCASHTRNXNS: integer (nullable = true)
 |-- AUTHCASHPERIOD: integer (nullable = true)
 |-- AUTHTEBONUS: double (nullable = true)
 |-- AUTHTMPBONUS: double (n

In [101]:
dfRiskdomainsMax=dfRiskdomains.where(to_date(col('REFDATE'))<'2017-11-01') \
    .where(col("NAME").isin({"Primary Group 1", "Additional Group 4","Personal Loan"})) \
    .groupBy('CUSTSERNO','NAME').agg(max('TM_INSERT').alias('FECHAMAX'))

dfRiskdomains2=dfRiskdomains.where(col("NAME").isin({"Primary Group 1", "Additional Group 4","Personal Loan"})) \
    .join(dfRiskdomainsMax,(dfRiskdomainsMax.CUSTSERNO==dfRiskdomains.CUSTSERNO) & (dfRiskdomainsMax.NAME==dfRiskdomains.NAME) & (dfRiskdomainsMax.FECHAMAX==dfRiskdomains.TM_INSERT),'leftsemi') \
    .select('CUSTSERNO','NAME','AUTHLIMIT','AUTHCASHLIMIT')
# dfStatementinstoffers2.printSchema()

In [102]:
dfRiskdomains2.persist().count()

20462249

In [103]:
dfRiskdomains2.printSchema()

root
 |-- CUSTSERNO: long (nullable = true)
 |-- NAME: string (nullable = true)
 |-- AUTHLIMIT: double (nullable = true)
 |-- AUTHCASHLIMIT: double (nullable = true)



In [104]:
dfRiskdomains2.select('CUSTSERNO').distinct().count()

6297269

In [105]:
dfPG1=dfRiskdomains2.where(col('NAME')=="Primary Group 1").withColumnRenamed('AUTHLIMIT','tmpLimitUse') \
    .withColumnRenamed('AUTHCASHLIMIT','tmpLimitCashOut').drop('NAME').dropDuplicates(['CUSTSERNO'])

In [106]:
dfPG1.select('CUSTSERNO').distinct().count()

4519582

In [107]:
dfPG1.printSchema()

root
 |-- CUSTSERNO: long (nullable = true)
 |-- tmpLimitUse: double (nullable = true)
 |-- tmpLimitCashOut: double (nullable = true)



In [108]:
dfAG4=dfRiskdomains2.where(col('NAME')=="Additional Group 4").withColumnRenamed('AUTHLIMIT','tmpLimitOffUs') \
    .drop('NAME').drop('AUTHCASHLIMIT').dropDuplicates(['CUSTSERNO'])

In [109]:
dfAG4.select('CUSTSERNO').distinct().count()

4827484

In [110]:
dfAG4.count()

4827484

In [111]:
dfPL=dfRiskdomains2.where(col('NAME')=="Personal Loan").withColumnRenamed('AUTHLIMIT','tmpLimitPersLoan') \
    .drop('NAME').drop('AUTHCASHLIMIT').dropDuplicates(['CUSTSERNO'])

In [112]:
dfPL.select('CUSTSERNO').distinct().count()

5579982

In [113]:
dfPL.count()

5579982

In [114]:
dfUniqueCust=dfRiskdomains2.select('CUSTSERNO').distinct()

In [115]:
dfPG1.printSchema()

root
 |-- CUSTSERNO: long (nullable = true)
 |-- tmpLimitUse: double (nullable = true)
 |-- tmpLimitCashOut: double (nullable = true)



In [116]:
dfLimTmp1=dfUniqueCust.join(dfPG1,dfUniqueCust.CUSTSERNO==dfPG1.CUSTSERNO, 'left_outer').drop(dfPG1.CUSTSERNO)

In [117]:
dfLimTmp1.show(2)

+---------+-----------+---------------+
|CUSTSERNO|tmpLimitUse|tmpLimitCashOut|
+---------+-----------+---------------+
|    13401|     -100.0|          -10.0|
|    67148|     -100.0|          -15.0|
+---------+-----------+---------------+
only showing top 2 rows



In [118]:
dfLimTmp2=dfLimTmp1.join(dfAG4,dfLimTmp1.CUSTSERNO==dfAG4.CUSTSERNO, 'left_outer').drop(dfAG4.CUSTSERNO)

In [119]:
dfLimTmp2.show(2)

+---------+-----------+---------------+-------------+
|CUSTSERNO|tmpLimitUse|tmpLimitCashOut|tmpLimitOffUs|
+---------+-----------+---------------+-------------+
|    13401|     -100.0|          -10.0|       -100.0|
|    67148|     -100.0|          -15.0|       -100.0|
+---------+-----------+---------------+-------------+
only showing top 2 rows



In [120]:
dfLimites=dfLimTmp2.join(dfPL,dfLimTmp2.CUSTSERNO==dfPL.CUSTSERNO, 'left_outer').drop(dfPL.CUSTSERNO).fillna(0)

In [121]:
dfLimites.persist().count()

6297269

In [122]:
dfDefAccLimitsTmp=dfDefAcc.join(dfLimites,dfLimites.CUSTSERNO==dfDefAcc.CUSTSERNO,'left_outer') \
    .drop(dfDefAcc.CUSTSERNO).fillna(0)

In [123]:
dfDefAcc.count()

3991780

In [124]:
dfDefAccLimitsTmp.persist().count()

3991780

In [125]:
# mostrarNulos(dfDefAccLimitsTmp).toPandas()

In [126]:
dfDefAccLimitsTmp.createOrReplaceTempView("tdfDefAccLimitsTmp")

In [127]:
dfDefAccLimits=spark.sql("""
    SELECT ACCOUNTPK, BALANCE, INSTALMENTBALANCEFOROVLMCHECK, CREDITLIMIT, LimitUse, LimitOffUs, LimitPersLoan, 
    CASE WHEN tmpLimitCashOut<0 THEN (LimitUse*tmpLimitCashOut/(-100)) ELSE tmpLimitCashOut END AS LimitCashOut
    FROM (SELECT ACCOUNTPK, BALANCE, INSTALMENTBALANCEFOROVLMCHECK, CREDITLIMIT, tmpLimitCashOut,
    CASE WHEN tmpLimitUse<0 THEN (CREDITLIMIT*tmpLimitUse/(-100)) ELSE tmpLimitUse END AS LimitUse,
    CASE WHEN tmpLimitOffUs<0 THEN (CREDITLIMIT*tmpLimitOffUs/(-100)) ELSE tmpLimitOffUs END AS LimitOffUs,
    CASE WHEN tmpLimitPersLoan<0 THEN (CREDITLIMIT*tmpLimitPersLoan/(-100)) ELSE tmpLimitPersLoan END AS LimitPersLoan
    FROM tdfDefAccLimitsTmp) A""")

In [94]:
# dfDefAccLimits.persist().describe().toPandas()

,summary,ACCOUNTPK,BALANCE,INSTALMENTBALANCEFOROVLMCHECK,CREDITLIMIT,LimitUse,LimitOffUs,LimitPersLoan,LimitCashOut
0,count,3991780,3991780,3991780,3991780,3991780,3991780,3991780,3991780
1,mean,8131360.095442635,-494.8582471979921,-563.7208977784344,3731.082978944235,3809.3147984101333,3432.1167758875467,754.773367319842,384.19598720987887
2,stddev,4308956.778108518,922.0532597219234,1334.881613843868,4161.313003661229,5281.76385666031,5291.929836213554,2825.46767215399,667.5731161227383
3,min,3,-39146.57,-34274.1,0.0,0.0,0.0,0.0,0.0
4,max,13627540,26608.24,57490.97,85000.0,50000.0,50000.0,114000.0,9000.0


In [128]:
dfDefAccLimits.printSchema()

root
 |-- ACCOUNTPK: long (nullable = true)
 |-- BALANCE: double (nullable = false)
 |-- INSTALMENTBALANCEFOROVLMCHECK: double (nullable = false)
 |-- CREDITLIMIT: double (nullable = false)
 |-- LimitUse: double (nullable = true)
 |-- LimitOffUs: double (nullable = true)
 |-- LimitPersLoan: double (nullable = true)
 |-- LimitCashOut: double (nullable = true)



In [129]:
dfDefAccLimits.persist().count()

3991780

### Final Matrix Union

In [130]:
dfFinTmp=dfDefAccLimits.join(dfClBalInstOff,dfDefAccLimits.ACCOUNTPK==dfClBalInstOff.ACC_SERNO,'left_outer').drop(dfClBalInstOff.ACC_SERNO)

In [131]:
dfFinTmp2=dfFinTmp.join(dfTsysBin, dfFinTmp.ACCOUNTPK==dfTsysBin.ACCOUNTPK,'left_outer').drop(dfTsysBin.ACCOUNTPK)

In [132]:
dfFinTmp2.persist().count()

3991780

In [133]:
dfFinTmp3=dfFinTmp2.join(dfOverlimitAuth, dfFinTmp2.ACCOUNTPK==dfOverlimitAuth.ROW_SERNO,'left_outer').drop(dfOverlimitAuth.ROW_SERNO)

In [134]:
len(dfFinTmp3.columns)

18

In [135]:
# mostrarNulos(dfFinTmp2).toPandas()

In [136]:
dfFinTmp4=dfFinTmp3.join(dfOffUs, dfFinTmp3.ACCOUNTPK==dfOffUs.ACCOUNTSERNOFK,'left_outer').drop(dfOffUs.ACCOUNTSERNOFK)

In [137]:
dfFinTmp4.count()

3991780

In [138]:
len(dfFinTmp4.columns)

487

In [144]:
dfFinalMatrix=dfFinTmp4.join(dfBuffer, dfFinTmp4.ACCOUNTPK==dfBuffer.ACCOUNTPK,'left_outer') \
    .drop(dfBuffer.ACCOUNTPK).withColumnRenamed('','NULO').fillna(0)

In [140]:
# dfFinalMatrix=dfDefAccLimits.join(dfClBalInstOff,dfDefAccLimits.ACCOUNTPK==dfClBalInstOff.ACC_SERNO,'left_outer').drop(dfClBalInstOff.ACC_SERNO) \
#     .join(dfTsysBin, dfDefAccLimits.ACCOUNTPK==dfTsysBin.ACCOUNTPK,'left_outer').drop(dfTsysBin.ACCOUNTPK) \
#     .join(dfOverlimitAuth, dfDefAccLimits.ACCOUNTPK==dfOverlimitAuth.ROW_SERNO,'left_outer').drop(dfOverlimitAuth.ROW_SERNO) \
#     .join(dfOffUs, dfDefAccLimits.ACCOUNTPK==dfOffUs.ACCOUNTSERNOFK,'left_outer').drop(dfOffUs.ACCOUNTSERNOFK) \
#     .join(dfBuffer, dfDefAccLimits.ACCOUNTPK==dfBuffer.ACCOUNTPK,'left_outer').drop(dfBuffer.ACCOUNTPK).fillna(0)

In [145]:
len(dfFinalMatrix.columns)

3188

In [142]:
dfFinalMatrix.count()

Py4JJavaError: An error occurred while calling o798.count.
: org.apache.spark.SparkException: Job 105 cancelled 
	at org.apache.spark.scheduler.DAGScheduler.org$apache$spark$scheduler$DAGScheduler$$failJobAndIndependentStages(DAGScheduler.scala:1435)
	at org.apache.spark.scheduler.DAGScheduler.handleJobCancellation(DAGScheduler.scala:1375)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.doOnReceive(DAGScheduler.scala:1622)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.onReceive(DAGScheduler.scala:1605)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.onReceive(DAGScheduler.scala:1594)
	at org.apache.spark.util.EventLoop$$anon$1.run(EventLoop.scala:48)
	at org.apache.spark.scheduler.DAGScheduler.runJob(DAGScheduler.scala:628)
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:1918)
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:1931)
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:1944)
	at org.apache.spark.SparkCont

In [108]:
dfFinalMatrix.printSchema()

root
 |-- ACCOUNTPK: long (nullable = true)
 |-- BALANCE: double (nullable = false)
 |-- INSTALMENTBALANCEFOROVLMCHECK: double (nullable = false)
 |-- CREDITLIMIT: double (nullable = false)
 |-- LimitUse: double (nullable = false)
 |-- LimitOffUs: double (nullable = false)
 |-- LimitPersLoan: double (nullable = false)
 |-- LimitCashOut: double (nullable = false)
 |-- CLOSING_BALANCE: double (nullable = false)
 |-- INSTALLMENTSOFFERS: long (nullable = true)
 |-- Emprestimo_Pessoal: long (nullable = true)
 |-- Pagamento_de_Contas: long (nullable = true)
 |-- Parcelamento_Fatura_Automatico: long (nullable = true)
 |-- Parcelamento_Fatura_Facil: long (nullable = true)
 |-- Saque_Parcelado: long (nullable = true)
 |-- Saque_a_Vista: long (nullable = true)
 |-- Overlimit: long (nullable = true)
 |-- Overlimit_Auth: integer (nullable = true)
 |-- NULO: double (nullable = false)
 |-- 0742: double (nullable = false)
 |-- 0763: double (nullable = false)
 |-- 0780: double (nullable = false)
 |-- 

### Dataframe for cluster analysis by department and sector

In [125]:
dfAnalysis=dfTemp2.groupBy("ACCOUNTPK","PRF_DS_DEPARTAMENTO","PRF_DS_SETOR").sum("PURCHASES").withColumnRenamed('sum(PURCHASES)',"PURCHASES")

In [126]:
dfAnalysis.printSchema()

root
 |-- ACCOUNTPK: long (nullable = true)
 |-- PRF_DS_DEPARTAMENTO: string (nullable = true)
 |-- PRF_DS_SETOR: string (nullable = true)
 |-- PURCHASES: double (nullable = true)



In [ ]:
#dfDemographic=dfAccounts.join(dfPeople,dfPeople.PEOPLESERNO==dfAccounts.PEOPLESERNOFK)

In [ ]:
# dfAnalysisBin=dfTemp2.groupBy("ACCOUNTPK","PRF_DS_DEPARTAMENTO","PRF_DS_SETOR") \
#     .agg(countDistinct(dfTemp2.PRF_CD_SUBFAMILIA).alias('SUBFAMILIES'))

In [ ]:
# dfAnalysisBin.printSchema()

### November validation dataframe

In [110]:
dfTrantsysTratada.printSchema()

root
 |-- TRANSACTIONSERNO: long (nullable = true)
 |-- ACCOUNTSERNOFK: long (nullable = true)
 |-- TYPESERNO_GLEDGER: long (nullable = true)
 |-- I013_TRXN_DATE: string (nullable = true)
 |-- I018_MERCH_TYPE: string (nullable = true)
 |-- AMOUNT: double (nullable = true)
 |-- VALUEDATE: string (nullable = true)
 |-- POSTDATE: string (nullable = true)
 |-- STGENERAL: string (nullable = true)
 |-- REFDATE: timestamp (nullable = true)
 |-- FG_CURRENT: integer (nullable = true)
 |-- TM_INSERT: timestamp (nullable = true)
 |-- PRODUTO: string (nullable = true)



In [111]:
dfValidacionNov=dfTrantsys.where((col('I013_TRXN_DATE')>='20171101') & (col('I013_TRXN_DATE')<'20171201')) \
    .join(dfCodeproducts,dfCodeproducts.SERNO==dfTrantsys.TYPESERNO_GLEDGER) \
    .where((col('TIPO_TRANSACAO')=='producao') & (col('FG_TRNS')=='TRANSACAO') & (col('PRODUTO')!='Off_Us')) \
    .select('TRANSACTIONSERNO','ACCOUNTSERNOFK','TYPESERNO_GLEDGER','VALUEDATE','I013_TRXN_DATE','POSTDATE','STGENERAL','REFDATE','FG_CURRENT','TM_INSERT','PRODUTO')
    #.where((col('VALUEDATE')>='20171101') & (col('VALUEDATE')<'20171101'))   .where(to_date(col('REFDATE'))>='2017-11-01') 
    #.select('TRANSACTIONSERNO','ACCOUNTSERNOFK','TYPESERNO_GLEDGER','I018_MERCH_TYPE','AMOUNT','VALUEDATE','POSTDATE','STGENERAL','REFDATE','FG_CURRENT','TM_INSERT','PRODUTO')

In [ ]:
dfValidacionNov=dfTrantsysTratada.drop('PRODUTO').where((col('I013_TRXN_DATE'))>='2017-11-01') \
    .join(dfCodeproducts,dfCodeproducts.SERNO==dfTrantsysTratada.TYPESERNO_GLEDGER) \
    .where((col('TIPO_TRANSACAO')=='producao') & (col('FG_TRNS')=='TRANSACAO') & (col('PRODUTO')!='Off_Us')) \
    .select('TRANSACTIONSERNO','ACCOUNTSERNOFK','TYPESERNO_GLEDGER','VALUEDATE','I013_TRXN_DATE','POSTDATE','STGENERAL','REFDATE','FG_CURRENT','TM_INSERT','PRODUTO')
    #.where((col('VALUEDATE')>='20171101') & (col('VALUEDATE')<'20171101'))   .where(to_date(col('REFDATE'))>='2017-11-01') 
    #.select('TRANSACTIONSERNO','ACCOUNTSERNOFK','TYPESERNO_GLEDGER','I018_MERCH_TYPE','AMOUNT','VALUEDATE','POSTDATE','STGENERAL','REFDATE','FG_CURRENT','TM_INSERT','PRODUTO')

In [112]:
dfValidacionNov.persist().count()

617879

In [113]:
dfValidacionNov.where(col('PRODUTO').like('Empr%')).show(5)

+----------------+--------------+-----------------+---------+--------------+--------+---------+--------------------+----------+--------------------+------------------+
|TRANSACTIONSERNO|ACCOUNTSERNOFK|TYPESERNO_GLEDGER|VALUEDATE|I013_TRXN_DATE|POSTDATE|STGENERAL|             REFDATE|FG_CURRENT|           TM_INSERT|           PRODUTO|
+----------------+--------------+-----------------+---------+--------------+--------+---------+--------------------+----------+--------------------+------------------+
|      2792324521|       6419151|             3328|     null|      20171129|    null|     IPST|2017-12-01 00:00:...|         1|2017-12-02 03:45:...|Emprestimo_Pessoal|
|      2792324580|       1554241|             3328|     null|      20171129|    null|     IPST|2017-12-01 00:00:...|         1|2017-12-02 03:45:...|Emprestimo_Pessoal|
|      2792324627|       1842232|             3328|     null|      20171129|    null|     IPST|2017-12-01 00:00:...|         1|2017-12-02 03:45:...|Emprestimo_P

In [114]:
dfValidacionNov.agg({'I013_TRXN_DATE':'min','POSTDATE':'min'}).show()

+-------------+-------------------+
|min(POSTDATE)|min(I013_TRXN_DATE)|
+-------------+-------------------+
|             |           20171101|
+-------------+-------------------+



In [115]:
dfValidacionNov.groupBy('PRODUTO').agg(countDistinct('ACCOUNTSERNOFK')).show(50)

+--------------------+------------------------------+
|             PRODUTO|count(DISTINCT ACCOUNTSERNOFK)|
+--------------------+------------------------------+
|     Saque_Parcelado|                          6501|
|  Emprestimo_Pessoal|                          1042|
|Parcelamento_Fatu...|                        136136|
|           Overlimit|                        169535|
| Pagamento_de_Contas|                         29631|
|       Saque_a_Vista|                          8034|
|Parcelamento_Fatu...|                         56201|
+--------------------+------------------------------+



### Save tables in hdfs

In [127]:
dfAnalysis.write.parquet("/analytics/bc1_clustering/tables/dfAnalysis02Ene")

In [ ]:
dfTsys.write.parquet("/analytics/bc1_clustering/tables/dfTsysAccYrOriginal")

In [ ]:
#dfEM.write.parquet("/analytics/bc1_clustering/tables/dfEM")
#dfBuffer.fillna(0).write.parquet("/tmp/dfBuffer")
#"7898940742035"
dfBuffer.fillna(0).write.parquet("/analytics/bc1_clustering/tables/dfBufferTest11Ene")
#dfBufferAccYrSubVal19Dic

In [146]:
dfFinalMatrix.write.parquet("/analytics/bc1_clustering/tables/dfFinalMatrixSubVal04Ene")

In [ ]:
dfEM.write.parquet("/analytics/bc1_clustering/tables/dfEM")

In [116]:
dfValidacionNov.write.parquet("/analytics/bc1_clustering/tables/dfValidacionNov")

In [143]:
dfBuffer=spark.read.parquet("/analytics/bc1_clustering/tables/dfBufferAccYrSubVal19Dic")

In [ ]:
dfFinalMatrix=spark.read.parquet("/analytics/bc1_clustering/tables/dfFinalMatrixGrpVal02Ene")

In [ ]:
dfTsys=spark.read.parquet("/analytics/bc1_clustering/tables/dfTsysAccYrOriginal")

In [1]:
pwd

'/var/sds/homes/fdeplasencia/workspace/bc1_clustering'